# IACS Capstone Project
## Team Spotify 1: Playlist Prediction
### Midterm 1: Data Exploration
#### Omar Abboud, Sonu Mehta, Laura Ware
This document contains preliminary data exploration on the Spotify API, specifically via the Python module "spotipy," which allows easy access to Spotify API data given a company-issued API key.

Examples from the Spotipy documentation were referenced for some of the below code:
https://spotipy.readthedocs.io/en/latest/

In [27]:
import numpy as np
import pandas as pd
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
import sys
from sklearn.metrics import mean_squared_error
from math import sqrt

### Authentication

In [3]:
client_credentials_manager = SpotifyClientCredentials(client_id='df846cfd28e745178054587b3484f91c', 
                                                      client_secret='e3d39fc92a954e028ff1490288f3fe5c')
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

### Playlist DataFrame Generation
The below code generates a DataFrame with all the playlists that were generated by the user ID "spotify" - in other words, those created by the company for wide distribution and recommendation. Alongside each playlist ID, we have extracted the mean popularity of the playlist's tracks, the number of followers of the playlist, as well as the playlist's name.

In [28]:
playlists = sp.user_playlists('spotify', limit=50, offset=0)
#playlists

In [31]:
names = []
total_tracks = []
followers = []
popularity_means = []
mean_num_markets = []
added_date = []
playlists = sp.user_playlists('spotify', limit=50, offset=0)

while playlists:
    for i, playlist in enumerate(playlists['items']):
        try:
            if i>9: break
            metadata = sp.user_playlist('spotify', playlist_id=playlist['id'],
                fields='followers.total,tracks.items(added_at,track.available_markets, track.popularity, track.name)')
            popularities = np.empty(len(metadata['tracks']['items']))
            added_at = []
            num_markets = np.empty(len(metadata['tracks']['items']))
            for index, item in enumerate(metadata['tracks']['items']): 
                popularities[index] = item['track']['popularity']
                added_at.append(item['added_at'])
                num_markets[index] = len(item['track']['available_markets'])
            added_date.append(added_at[0])
            mean_num_markets.append(num_markets.mean())
            popularity_means.append(popularities.mean())
            followers.append(metadata['followers']['total'])
            names.append(playlist['name'])
            total_tracks.append(playlist['tracks']['total'])
            print("%4d %s" % (i + 1 + playlists['offset'], playlist['name']))
        except:
            a=0
            print("NO METADATA")
    break
    if playlists['next']:
        playlists = sp.next(playlists)
    else:
        playlists = None
        break

   1 Today's Top Hits
   2 Rap Caviar
   3 electroNOW
   4 Afternoon Acoustic
   5 Peaceful Piano
   6 TGIF
   7 From Beginning To End:  Johnny Cash
   8 This Is:  Johnny Cash
   9 The Refugee Playlist
  10 This Is: AC/DC


In [32]:
playlist_data = pd.DataFrame({
     'names': names,
     'total_tracks': total_tracks,
     'followers': followers,
     'mean_popularity': popularity_means,
    'added_date': added_date, 
    'num_markets': mean_num_markets
    
    })

In [33]:
followers_array = np.array(followers)
followers_array.mean()
followers_array.shape

(10,)

### Baseline Model #1 - Guessing the mean number of followers

In [20]:
playlist_data['followers'].mean()

656948.78

In [50]:
def getRMSE(predicted, actual):
    return np.sqrt(((predicted - actual) ** 2).mean())
def testRMSE(predicted, actual):
    return sqrt(mean_squared_error(actual, predicted))

In [51]:
predictions = np.ones((50,1))*followers_array.mean()
print(getRMSE(predictions, followers_array))
print(testRMSE(predictions, followers_array))

2118484.84157
2118484.841571224


### Baseline Model #2 - Linear Regression based on mean popularity

In [53]:
from sklearn.linear_model import LinearRegression as LinReg

In [60]:
popularity_means_array = np.array(popularity_means).shape

In [34]:
#model = LinReg()
#model.fit(popularity_means_array, followers)

In [35]:
playlist_data.head()

,added_date,followers,mean_popularity,names,num_markets,total_tracks
0,2017-02-24T20:39:34Z,13528304,76.560000,Today's Top Hits,30.160000,50
1,2017-02-24T13:23:06Z,5733505,61.140000,Rap Caviar,43.240000,50
2,2017-02-24T05:50:57Z,3905547,57.160000,electroNOW,47.540000,50
3,2017-02-15T21:35:55Z,2449980,51.569444,Afternoon Acoustic,33.902778,72
4,2017-02-17T17:04:20Z,2583852,56.770000,Peaceful Piano,45.070000,150


In [25]:
playlist_data.to_csv('playlist_data.csv',encoding='utf-8')

### Track DataFrame Generation
In the below code, we perform a deeper analysis of one sample playlist, "Today's Top Hits." We obtain the list of tracks from the playlist and make an API call to obtain more detailed information about each track, particularly audio features. We also include a "sequence" column that indicates where in the playlist that particular track is located.

In [38]:
sp.user_playlist('spotify','5FJXhjdILmRA2z5bvz4nzf')['tracks']['items'][0]

{'added_at': '2017-02-24T20:39:34Z',
 'added_by': {'external_urls': {'spotify': 'http://open.spotify.com/user/spotify'},
  'href': 'https://api.spotify.com/v1/users/spotify',
  'id': 'spotify',
  'type': 'user',
  'uri': 'spotify:user:spotify'},
 'is_local': False,
 'track': {'album': {'album_type': 'single',
   'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/23fqKkggKUBHNkbKtXEls4'},
     'href': 'https://api.spotify.com/v1/artists/23fqKkggKUBHNkbKtXEls4',
     'id': '23fqKkggKUBHNkbKtXEls4',
     'name': 'Kygo',
     'type': 'artist',
     'uri': 'spotify:artist:23fqKkggKUBHNkbKtXEls4'},
    {'external_urls': {'spotify': 'https://open.spotify.com/artist/0C8ZW7ezQVs4URX5aX7Kqx'},
     'href': 'https://api.spotify.com/v1/artists/0C8ZW7ezQVs4URX5aX7Kqx',
     'id': '0C8ZW7ezQVs4URX5aX7Kqx',
     'name': 'Selena Gomez',
     'type': 'artist',
     'uri': 'spotify:artist:0C8ZW7ezQVs4URX5aX7Kqx'}],
   'available_markets': ['US'],
   'external_urls': {'spotify': '

In [79]:
playlist_sample = sp.user_playlist('spotify','5FJXhjdILmRA2z5bvz4nzf')['tracks']['items']
list_of_tracks = []
for a in playlist_sample:
    list_of_tracks.append(a['track'])
sample = pd.DataFrame(list_of_tracks)[['id','name','external_ids','artists',
                                       'duration_ms','explicit','track_number','popularity']]
features = sp.audio_features(tracks=sample['id'])
features_df = pd.DataFrame(features)
sample['acousticness'] = features_df['acousticness']
sample['danceability'] = features_df['danceability']
sample['energy'] = features_df['energy']
sample['instrumentalness'] = features_df['instrumentalness']
sample['key'] = features_df['key']
sample['liveness'] = features_df['liveness']
sample['loudness'] = features_df['loudness']
sample['mode'] = features_df['speechiness']
sample['tempo'] = features_df['tempo']
sample['time_signature'] = features_df['time_signature']
sample['valence'] = features_df['valence']
sample['sequence'] = sample.index + 1
sample.head()

,id,name,external_ids,artists,duration_ms,explicit,track_number,popularity,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,tempo,time_signature,valence,sequence
0,1dNIEtp7AY3oDAKCGg2XkH,Something Just Like This,{u'isrc': u'USQX91700278'},"[{u'name': u'The Chainsmokers', u'external_url...",247626,False,1,0,0.0306,0.607,0.649,0.000025,11,0.1740,-6.695,0.0362,102.996,4,0.470,1
1,12GEpg2XOPyqk03JZEZnJs,It Ain’t Me (with Selena Gomez),{u'isrc': u'SEBGA1700015'},"[{u'name': u'Kygo', u'external_urls': {u'spoti...",220780,False,1,76,0.0905,0.648,0.532,0.000000,0,0.0831,-6.597,0.0746,99.983,4,0.497,2
2,6AeQlMyRzvSl1nkFztZyKl,Issues,{u'isrc': u'USUM71615691'},"[{u'name': u'Julia Michaels', u'external_urls'...",176346,False,1,82,0.4160,0.704,0.423,0.000000,8,0.0607,-6.792,0.0862,113.962,4,0.450,3
3,0FE9t6xYkqWXU2ahLh6D8X,Shape of You,{u'isrc': u'GBAHS1600463'},"[{u'name': u'Ed Sheeran', u'external_urls': {u...",233712,False,1,100,0.5810,0.825,0.652,0.000000,1,0.0931,-3.183,0.0802,95.977,4,0.933,4
4,3ebXMykcMXOcLeJ9xZ17XH,Scared To Be Lonely,{u'isrc': u'NLM5S1600025'},"[{u'name': u'Martin Garrix', u'external_urls':...",220883,False,1,91,0.0895,0.584,0.540,0.000000,1,0.2610,-7.786,0.0576,137.972,4,0.190,5


In [81]:
sample.to_csv('sample.csv',encoding='utf-8')